In [1]:
import lz4.frame as lz4f
import pickle
import json
import time
import cloudpickle
import gzip
import os
from optparse import OptionParser

import uproot
import numpy as np
from coffea import hist, processor
from coffea.util import load, save

import sys
sys.path.append('./analysis/topEFT')

import topeft
from topcoffea.modules import samples
from topcoffea.modules.fileReader import GetFiles, GetAllInfoFromFile
outname = 'plotsTopEFT'

nworkers = 8

In [2]:
cfgfile=('cfg/samples.cfg')

In [3]:
#change place
fileset = {'Temp' : ["root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root"]}

In [4]:
path = fileset['Temp'][0]

In [5]:
path

'root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root'

In [6]:
def GetOptions(path, sample, options = ""):
  if not path.endswith('/'): path += '/'
  if not sample.endswith(".root"): sample += '.root'
  #doPUweight  = 'PUweight,' if IsVarInTree(path+sample, 'puWeight') else ''
  #doJECunc    = 'JECunc,'   if IsVarInTree(path+sample, 'Jet_pt_jesTotalUp') else ''
  #doIFSR      = 'doIFSR,'   if IsVarInTree(path+sample, 'nPSWeight') and GetValOfVarInTree(path+sample, 'nPSWeight') == 4 else ''
  #useJetPtNom = 'JetPtNom,' if IsVarInTree(path+sample, 'Jet_pt_nom') else ''
  #options += doPUweight + doJECunc + doIFSR + useJetPtNom + options
  if options.endswith(','): options = options[:-1]
  return options

In [7]:
#rewritten from topcoffea/topcoffea/modules/samples.py
def getdictionary(cfgfile, pretend=1,test=1,verbose=1,path='',sample='',xsec='xsec',year=-1,options='',treeName='Events'):
    samplefiles = {}
    fileopt = {}
    xsecdic = {}
    sampdic = {}
    f = open(cfgfile)
    lines = f.readlines()
    for l in lines:
        l = l.replace(' ', '')
        l = l.replace('\n', '')
        if l.startswith('#'): 
            continue
        if '#' in l: l = l.split('#')[0]
        if l == '': continue
        if l.endswith(':'): l = l[:-1]
        if not ':' in l:
            if l in ['path', 'verbose', 'pretend', 'test', 'options', 'xsec', 'year', 'treeName']: continue
            else: samplefiles[l]=l
        else:
            lst = l.split(':')
            key = lst[0]
            val = lst[1] if lst[1] != '' else lst[0]
            if   key == 'pretend'   : pretend   = 1
            elif key == 'verbose'   : verbose   = int(val) if val.isdigit() else 1
            elif key == 'test'      : dotest    = 1
            elif key == 'path'      :
                path = val
                if len(lst) > 2: 
                    for v in lst[2:]: path += ':'+v
            elif key == 'options'   : options   = val
            elif key == 'xsec'      : xsec      = val
            elif key == 'year'      : year      = int(val)
            elif key == 'treeName'  : treeName  = val
            else:
                fileopt[key] = options
                if len(lst) >= 3: fileopt[key] += lst[2]
                samplefiles[key] = val
    for sname in samplefiles.keys():
        sampdic[sname] = {}
        #sampdic[sname]['files']      = GetFiles(path, samplefiles[sname])
        sampdic[sname]['files']      = path
        extraOption = GetOptions(path, sampdic[sname]['files'][0].split('/')[-1])
        sampdic[sname]['options']    = fileopt[sname] + ', ' + extraOption
        sampdic[sname]['xsec']       = xsecdic[sname] if sname in xsecdic.keys() else 1
        sampdic[sname]['year']       = year
        sampdic[sname]['treeName']   = treeName
        nEvents, nGenEvents, nSumOfWeights, isData = GetAllInfoFromFile(sampdic[sname]['files'], sampdic[sname]['treeName'])
        sampdic[sname]['nEvents']       = nEvents
        sampdic[sname]['nGenEvents']    = nGenEvents
        sampdic[sname]['nSumOfWeights'] = nSumOfWeights
        sampdic[sname]['isData']        = isData        
    return(sampdic)

#   skipping this, not sure if important
#   Re-assign arguments...
#   aarg = sys.argv
#   if '--pretend' in aarg or '-p' in aarg : pretend     = args.pretend
#   if '--test'    in aarg or '-t' in aarg : dotest      = args.test
#   if args.path       != ''       : path        = args.path
#   if args.options    != ''       : options     = args.options
#   if args.xsec       != 'xsec'   : xsec        = args.xsec
#   if args.year       != -1       : year        = args.year
#   if args.treeName   != 'Events' : treeName    = args.treeName
#   if args.verbose    != 0        : verbose     = int(args.verbose)
#   xsecdic = loadxsecdic(xsec, verbose)


In [8]:
samplesdict = getdictionary(cfgfile,path=path)

In [9]:
flist = {}; xsec = {}; sow = {}; isData = {}
for k in samplesdict.keys():
    flist[k] = [samplesdict[k]['files']]
    xsec[k]  = samplesdict[k]['xsec']
    sow[k]   = samplesdict[k]['nSumOfWeights']
    isData[k]= samplesdict[k]['isData']

In [10]:
processor_instance = topeft.AnalysisProcessor(samplesdict)

In [11]:
flist

{'DYJetsToLL_M_10to50': ['root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root'],
 'ZZTo2L2Nu': ['root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root'],
 'ZZTo4L': ['root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root'],
 'WZTo3LNU': ['root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root'],
 'WWTo2L2Nu': ['root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root'],
 'tW_noFullHad': ['root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root'],
 'tbarW_noFullHad': ['root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root'],
 'TT': ['root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/c

In [13]:
samplesdict

{'DYJetsToLL_M_10to50': {'files': 'root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root',
  'options': ', ',
  'xsec': 1,
  'year': -1,
  'treeName': 'Events',
  'nEvents': 614400,
  'nGenEvents': 614400.0,
  'nSumOfWeights': 614400,
  'isData': False},
 'ZZTo2L2Nu': {'files': 'root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root',
  'options': ', ',
  'xsec': 1,
  'year': -1,
  'treeName': 'Events',
  'nEvents': 614400,
  'nGenEvents': 614400.0,
  'nSumOfWeights': 614400,
  'isData': False},
 'ZZTo4L': {'files': 'root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/coffea-casa/38E83594-51BD-7D46-B96D-620DD60078A7.root',
  'options': ', ',
  'xsec': 1,
  'year': -1,
  'treeName': 'Events',
  'nEvents': 614400,
  'nGenEvents': 614400.0,
  'nSumOfWeights': 614400,
  'isData': False},
 'WZTo3LNU': {'files': 'root://hcc-stash.unl.edu:1094//osgconnect/public/dweitzel/co

In [ ]:
# Run the processor and get the output
tstart = time.time()
#output = processor.run_uproot_job(flist, treename='Events', processor_instance=processor_instance, executor=processor.futures_executor, executor_args={'workers': nworkers, 'pre_workers': 1}, chunksize=500000)
#output = processor.run_uproot_job(flist, treename='Events', processor_instance=processor_instance, executor=processor.futures_executor, executor_args={'nano':True,'workers': nworkers, 'pre_workers': 1}, chunksize=500000)
output = processor.run_uproot_job(flist, treename='Events', processor_instance=processor_instance, executor=processor.futures_executor, executor_args={'nano':True}, chunksize=500000)

dt = time.time() - tstart

nbins = sum(sum(arr.size for arr in h._sumw.values()) for h in output.values() if isinstance(h, hist.Hist))
nfilled = sum(sum(np.sum(arr > 0) for arr in h._sumw.values()) for h in output.values() if isinstance(h, hist.Hist))
print("Filled %.0f bins, nonzero bins: %1.1f %%" % (nbins, 100*nfilled/nbins,))
print("Processing time: %1.2f s with %i workers (%.2f s cpu overall)" % (dt, nworkers, dt*nworkers, ))

# This is taken from the DM photon analysis...
# Pickle is not very fast or memory efficient, will be replaced by something better soon
#    with lz4f.open("pods/"+options.year+"/"+dataset+".pkl.gz", mode="xb", compression_level=5) as fout:
os.system("mkdir -p histos/")
print('Saving output in %s...'%("histos/" + outname + ".pkl.gz"))
with gzip.open("histos/" + outname + ".pkl.gz", "wb") as fout:
    cloudpickle.dump(output, fout)
print('Done!')



/opt/conda/lib/python3.8/site-packages/coffea/util.py:98: Awkward0Warning: In coffea version 0.7 (target date: January 2021), this will be an error.
(Set awkward1.deprecations_as_errors = True to get a stack trace now.)
TypeError: <class 'coffea.nanoaod.nanoevents.NanoEvents'> relies exclusively on awkward 0.x and will be removed in upcoming versions of coffea!
  warnings.warn(message, Awkward0Warning)
/opt/conda/lib/python3.8/site-packages/coffea/util.py:98: Awkward0Warning: In coffea version 0.7 (target date: January 2021), this will be an error.
(Set awkward1.deprecations_as_errors = True to get a stack trace now.)
TypeError: <class 'coffea.nanoaod.nanoevents.NanoCollection'> relies exclusively on awkward 0.x and will be removed in upcoming versions of coffea!
  warnings.warn(message, Awkward0Warning)
